In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import IPython
import subprocess
import time
import os 
import glob

In [2]:
endpointURL_reactome_v87 = "http://localhost:3030/reactome_v87/query"
endpointURL_panther_standalone_biopax = "http://localhost:3030/panther_standalone_biopax/query"
endpointURL_reactome_pc = "http://localhost:3030/reactome_pc/query"
endpointURL_panther_pc = "http://localhost:3030/panther_pc/query"
endpointURL_pathbank_pc = "http://localhost:3030/pathbank_pc/query"
endpointURL_humancyc_pc = "http://localhost:3030/humancyc_pc/query"
endpointURL_kegg_pc = "http://localhost:3030/kegg_pc/query"
endpointURL_pid_pc = "http://localhost:3030/pid_pc/query"
endpointURL_inoh_pc = "http://localhost:3030/inoh_pc/query"
endpointURL_netpath_pc = "http://localhost:3030/netpath_pc/query"
rdfFormat = "turtle"
current_directory = os.getcwd()
BioPAX_Ontology_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/BioPAX_Ontology', 'biopax-level3.owl')

In [3]:
reactomeVersion = 87
prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX chebi: <http://purl.obolibrary.org/obo/chebi/>
PREFIX chebidb: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX chebirel: <http://purl.obolibrary.org/obo/CHEBI#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

PREFIX bp3: <http://www.biopax.org/release/biopax-level3.owl#>

# Homo_sapiens-20170221.owl
#PREFIX reactome: <http://www.reactome.org/biopax/59/48887#> 
#
# Homo_sapiens-20210608.owl
#PREFIX reactome: <http://www.reactome.org/biopax/77/48887#>
#
# Homo_sapiens-20220614.owl
#PREFIX reactome: <http://www.reactome.org/biopax/81/48887#>
#
# Homo_sapiens-20221130.owl
#PREFIX reactome: <http://www.reactome.org/biopax/83/48887#>

PREFIX reactome: <http://www.reactome.org/biopax/{}/48887#>
""".format(reactomeVersion)

biopaxURI = "http://www.biopax.org/release/biopax-level3.owl#"

In [4]:
def displaySparqlResults(results):
    '''
    Displays as HTML the result of a SPARQLWrapper query in a Jupyter notebook.
    
        Parameters:
            results (dictionnary): the result of a call to SPARQLWrapper.query().convert()
    '''
    variableNames = results['head']['vars']
    tableCode = '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format('</th><th>'.join(variableNames), '</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join([row[vName]['value'] if vName in row.keys() else "&nbsp;" for vName in variableNames]))for row in results["results"]["bindings"]))
    IPython.display.display(IPython.display.HTML(tableCode))

## Mappings to UniProt

In [5]:
# First SPARQL query using the string "UniProt" to map to UniProt resources
# BioPAX standalone export, Panther standalone, Reactome PC, Panther PC
query1 = """
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type bp3:UnificationXref .
            ?protein_ref_xref bp3:db "UniProt" .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:xref ?protein_ref_xref .
                ?protein_ref_xref rdf:type bp3:UnificationXref .
                ?protein_ref_xref bp3:db "UniProt" .
                ?protein_ref_xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "UniProt" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric
"""

# Query for Panther Standalone and Panther PC
query2 = """
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type/(rdfs:subClassOf*) bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type/(rdfs:subClassOf*) bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type/(rdfs:subClassOf*) bp3:ProteinReference .
            ?protein_ref bp3:memberEntityReference ?member_entity_ref .
            ?member_entity_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type/(rdfs:subClassOf*) bp3:UnificationXref .
            ?protein_ref_xref bp3:db "uniprot" .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:memberEntityReference ?member_entity_ref .
                ?member_entity_ref bp3:xref ?xref .
                ?xref rdf:type/(rdfs:subClassOf*) bp3:UnificationXref .
                ?xref bp3:db "uniprot" .
                ?xref bp3:id ?up_id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type/(rdfs:subClassOf*) bp3:UnificationXref .
            ?unif_xref bp3:db "uniprot" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric
"""

# HumanCyc PC, KEGG PC, PID PC
query3 = """  
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type bp3:UnificationXref .
            ?protein_ref_xref bp3:db "UniProt" .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:xref ?xref .
                ?xref rdf:type bp3:UnificationXref .
                ?xref bp3:db "UniProt" .
                ?xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "UniProt" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric
"""


# INOH PC
query4 = """
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_that_map_to_uniprot bp3:xref ?protein_xref .
            ?protein_xref rdf:type bp3:UnificationXref .
            ?protein_xref bp3:db "UniProt" .
            ?protein_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:entityReference .
            FILTER NOT EXISTS {
                ?protein_no_map_to_uniprot bp3:xref ?protein_xref .
                ?protein_xref rdf:type bp3:UnificationXref .
                ?protein_xref bp3:db "UniProt" .
                ?protein_xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "UniProt" .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric
"""

# NetPath PC
query5 = """ 
SELECT ?metric ?count
# Number of instances of Protein
WHERE {
OPTIONAL {
    {
        SELECT ?metric (COUNT(DISTINCT ?protein) AS ?count)
        WHERE {
            BIND("Protein" AS ?metric)
            ?protein rdf:type/(rdfs:subClassOf*) bp3:Protein .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_ref) AS ?count)
        WHERE {
            BIND("ProteinReference" AS ?metric)
            ?protein_ref rdf:type bp3:ProteinReference .
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_without_ref) AS ?count)
        WHERE {
            BIND("Protein with no PR" AS ?metric)
            OPTIONAL {
            ?protein_without_ref rdf:type/(rdfs:subClassOf*) bp3:Protein .
            FILTER NOT EXISTS {
                ?protein_without_ref bp3:entityReference ?ref .
                ?ref rdf:type bp3:ProteinReference .
            }
        }
        }
        GROUP BY ?metric
    }
    UNION
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_that_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Uniprot mappings" AS ?metric)
            ?protein_that_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_that_map_to_uniprot bp3:entityReference ?protein_ref .
            ?protein_ref rdf:type bp3:ProteinReference .
            ?protein_ref bp3:xref ?protein_ref_xref .
            ?protein_ref_xref rdf:type bp3:UnificationXref .
            ?protein_ref_xref bp3:db "UniProt"^^<xsd:string> .
            ?protein_ref_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?protein_no_map_to_uniprot) AS ?count)
        WHERE {
            BIND("Protein with no mapping" AS ?metric)
            ?protein_no_map_to_uniprot rdf:type/(rdfs:subClassOf*) bp3:Protein .
            ?protein_no_map_to_uniprot bp3:entityReference ?protein_ref .
            FILTER NOT EXISTS {
                ?protein_ref bp3:xref ?protein_ref_xref .
                ?protein_ref_xref rdf:type bp3:UnificationXref .
                ?protein_ref_xref bp3:db "UniProt"^^<xsd:string> .
                ?protein_ref_xref bp3:id ?id .
            }
        }
        GROUP BY ?metric
    }
    UNION 
    {
        SELECT ?metric (COUNT(DISTINCT ?up_id) AS ?count)
        WHERE {
            BIND("Unique Uniprot" AS ?metric)
            ?unif_xref rdf:type bp3:UnificationXref .
            ?unif_xref bp3:db "UniProt"^^<xsd:string> .
            ?unif_xref bp3:id ?up_id .
        }
        GROUP BY ?metric
    }
   }
}
ORDER BY ?metric
"""

TODO: change the path to fuseki server and BioPAX files + BioPAX ontology in the command line to launch the SPARQL endpoint

#### Query on Reactome BioPAX standalone export version 87

In [ ]:
ReactomeBioPAX_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/ReactomeBioPAX/', 'Homo_sapiens.owl')
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', ReactomeBioPAX_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/reactome_v87']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_v87)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_v87_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on Panther standalone BioPAX export

In [ ]:
PathwayCommons_Panther_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PantherBioPAX/BioPAX/')
Panther_owl_files = glob.glob(os.path.join(PathwayCommons_Panther_file_path, '*.owl'))

# remove incorrect owl files and add the corrected ones
to_remove = os.path.join(PathwayCommons_Panther_file_path, 'Gonadotropin_releasing_hormone_receptor_pathway.owl')
Panther_owl_files.remove(to_remove)
to_add = os.path.join(PathwayCommons_Panther_file_path, 'Gonadotropin_releasing_hormone_receptor_pathway_corrected.owl')
Panther_owl_files.append(to_add)
print(len(Panther_owl_files))

command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Panther_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/panther_standalone_biopax']

print(command)

process = subprocess.Popen(command)
time.sleep(60)
# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_panther_standalone_biopax)
sparql.setQuery(prefixes+query2) 

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export results to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/panther_standalone_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of Reactome from PathwayCommons

In [ ]:
PathwayCommons_Reactome_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/reactome_pc/', 'Homo_sapiens.owl')
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', PathwayCommons_Reactome_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/reactome_pc']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_reactome_pc)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open(f"../Results/reactome_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process    
process.kill()
time.sleep(60)

#### Query on BioPAX export of Panther from PathwayCommons

In [ ]:
PathwayCommons_Panther_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/panther_pc/')
Panther_owl_files = glob.glob(os.path.join(PathwayCommons_Panther_file_path, '*.owl'))

# remove incorrect owl files and add the corrected ones
to_remove = os.path.join(PathwayCommons_Panther_file_path, 'Gonadotropin_releasing_hormone_receptor_pathway.owl')
Panther_owl_files.remove(to_remove)
to_add = os.path.join(PathwayCommons_Panther_file_path, 'Gonadotropin_releasing_hormone_receptor_pathway_corrected.owl')
Panther_owl_files.append(to_add)
print(len(Panther_owl_files))


command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Panther_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/panther_pc']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_panther_pc)
sparql.setQuery(prefixes+query2)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/panther_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of PathBank from PathwayCommons

In [ ]:
# Update the PathwayCommons Pathbank OWL files
PathwayCommons_Pathbank_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/pathbank_pc/')
Pathbank_owl_files = glob.glob(os.path.join(PathwayCommons_Pathbank_file_path, '*.owl'))

# Remove the incorrect owl file and add the corrected one
to_remove1 = os.path.join(PathwayCommons_Pathbank_file_path, 'PW123534.owl')
to_remove2 = os.path.join(PathwayCommons_Pathbank_file_path, 'PW002037.owl')
Pathbank_owl_files.remove(to_remove1)
Pathbank_owl_files.remove(to_remove2)
to_add1 = os.path.join(PathwayCommons_Pathbank_file_path, 'PW123534_corrected.owl')
to_add2 = os.path.join(PathwayCommons_Pathbank_file_path, 'PW002037_corrected.owl')
Pathbank_owl_files.append(to_add1)
Pathbank_owl_files.append(to_add2)

command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Pathbank_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/pathbank_pc']


process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pathbank_pc)
sparql.setQuery(prefixes+query1)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pathbank_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of HumanCyc from PathwayCommons

In [ ]:
PathwayCommons_Humancyc_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/humancyc_pc/', 'biopax-level3.owl')
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', PathwayCommons_Humancyc_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/humancyc_pc']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_humancyc_pc)
sparql.setQuery(prefixes+query3)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/humancyc_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of KEGG from PathwayCommons

In [ ]:
PathwayCommons_Kegg_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/kegg_pc/')
Kegg_owl_files = glob.glob(os.path.join(PathwayCommons_Kegg_file_path, '*.owl'))
command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Kegg_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/kegg_pc']


process = subprocess.Popen(command)
time.sleep(60)query5

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_kegg_pc)
sparql.setQuery(prefixes+query3)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/kegg_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of PID from PathwayCommons

In [ ]:
PathwayCommons_Pid_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/pid_pc/NCI-Nature-Curated-final.bp3.owl')
command = [
    '/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',
    '--file', PathwayCommons_Pid_file_path,
    '--file', BioPAX_Ontology_file_path,
    '/pid_pc']
process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_pid_pc)
sparql.setQuery(prefixes+query3)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/pid_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

#### Query on BioPAX export of INOH from PathwayCommons

In [7]:
PathwayCommons_Inoh_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/inoh_pc/')
Inoh_owl_files_metabolic = glob.glob(os.path.join(PathwayCommons_Inoh_file_path, 'metabolic/*.owl'))
Inoh_owl_files_signaling = glob.glob(os.path.join(PathwayCommons_Inoh_file_path, 'signaling/*.owl'))
Inoh_owl_files = Inoh_owl_files_metabolic + Inoh_owl_files_signaling
command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Inoh_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/inoh_pc']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_inoh_pc)
sparql.setQuery(prefixes+query4)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/inoh_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)

17:23:44 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/../../BioPAX_Data/PathwayCommons/inoh_pc/metabolic/Citrate_cycle.owl
17:23:45 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/../../BioPAX_Data/PathwayCommons/inoh_pc/metabolic/Inositol_phosphate_metabolism.owl
17:23:45 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/../../BioPAX_Data/PathwayCommons/inoh_pc/metabolic/Glycine_Serine_metabolism.owl
17:23:45 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Review/BioPAXReview2024Codes/Figure4-5/Scripts/../../BioPAX_Data/PathwayCommons/inoh_pc/metabolic/Tryptophan_degradation.owl
17:23:45 INFO  Server          :: Dataset: in-memory: load file: /home/cbeust/Projects/2024/BioPAX_Revi

metric,count
Protein,12804
Protein with no PR,0
ProteinReference,1337
Uniprot mappings,12764
Unique Uniprot,3398


17:24:45 INFO  Fuseki          :: [8] GET http://localhost:3030/inoh_pc/query?query=%0APREFIX+rdf%3A+%3Chttp%3A//www.w3.org/1999/02/22-rdf-syntax-ns%23%3E%0APREFIX+rdfs%3A%3Chttp%3A//www.w3.org/2000/01/rdf-schema%23%3E%0APREFIX+owl%3A+%3Chttp%3A//www.w3.org/2002/07/owl%23%3E%0APREFIX+xsd%3A+%3Chttp%3A//www.w3.org/2001/XMLSchema%23%3E%0APREFIX+dc%3A+%3Chttp%3A//purl.org/dc/elements/1.1/%3E%0APREFIX+dcterms%3A+%3Chttp%3A//purl.org/dc/terms/%3E%0A%0APREFIX+chebi%3A+%3Chttp%3A//purl.obolibrary.org/obo/chebi/%3E%0APREFIX+chebidb%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI_%3E%0APREFIX+chebirel%3A+%3Chttp%3A//purl.obolibrary.org/obo/CHEBI%23%3E%0APREFIX+oboInOwl%3A+%3Chttp%3A//www.geneontology.org/formats/oboInOwl%23%3E%0A%0APREFIX+bp3%3A+%3Chttp%3A//www.biopax.org/release/biopax-level3.owl%23%3E%0A%0A%23+Homo_sapiens-20170221.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome.org/biopax/59/48887%23%3E+%0A%23%0A%23+Homo_sapiens-20210608.owl%0A%23PREFIX+reactome%3A+%3Chttp%3A//www.reactome

#### Query on BioPAX export of NetPath from PathwayCommons

In [ ]:
PathwayCommons_Netpath_file_path = os.path.join(current_directory, '../..', 'BioPAX_Data/PathwayCommons/netpath_pc/')

Netpath_owl_files = glob.glob(os.path.join(PathwayCommons_Netpath_file_path, '*.owl'))


# Remove the incorrect owl file and add the corrected one
to_remove1 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_7.owl')
to_remove2 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_11.owl')
to_remove3 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_22.owl')
to_remove4 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_9.owl')
Netpath_owl_files.remove(to_remove1)
Netpath_owl_files.remove(to_remove2)
Netpath_owl_files.remove(to_remove3)
Netpath_owl_files.remove(to_remove4)
to_add1 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_7_corrected.owl')
to_add2 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_11_corrected.owl')
to_add3 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_22_corrected.owl')
to_add4 = os.path.join(PathwayCommons_Netpath_file_path, 'NetPath_9_corrected.owl')
Netpath_owl_files.append(to_add1)
Netpath_owl_files.append(to_add2)
Netpath_owl_files.append(to_add3)
Netpath_owl_files.append(to_add4)

command = ['/home/cbeust/Softwares/JenaFuseki/apache-jena-fuseki-4.9.0/fuseki-server',]

for owl_file in Netpath_owl_files:
    command += ['--file'] 
    command += [f'{owl_file}']
command += ['--file']
command += [f'{BioPAX_Ontology_file_path}']
command += ['/netpath_pc']

process = subprocess.Popen(command)
time.sleep(60)

# execute SPARQL query
sparql = SPARQLWrapper(endpointURL_netpath_pc)
sparql.setQuery(prefixes+query5)

# display results
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
displaySparqlResults(results)

# export to CSV
sparql.setReturnFormat(CSV)
results = sparql.query().convert()
with open("../Results/netpath_pc_mappings_uniprot.csv", "wb") as f:
    f.write(results)

# end process
process.kill()
time.sleep(60)